In [33]:
import pandas as pd
import numpy as np
import rapidfuzz

In [34]:
state = 'CT'
occs_and_debt = pd.read_excel(r"C:\Users\david\OneDrive\Documents\GitHub\SPEOC-pt-1\Data\Post1790\CT\CT_post1790_CD_ledger.xlsx",
                             header=13, usecols='K, L, N, O, AD, AE, AT, AU') #occupations plus continental debt for conneticut

In [35]:
occs_and_debt = occs_and_debt[occs_and_debt['State'].str.contains(state) == True].drop('State', axis=1)
print(occs_and_debt)

     Occupation  Dollar  Cents  Dollar.1  Cents.1  Dollar.2  Cents.2
0      Merchant  1064.0   75.0     532.0     37.0     508.0     51.0
1        Farmer   449.0   96.0     224.0     97.0     232.0     10.0
3      Merchant   196.0   75.0      98.0     37.0     172.0     24.0
4      Merchant    53.0   58.0      26.0     79.0      67.0      6.0
5           NaN   345.0   50.0     172.0     75.0     248.0     76.0
...         ...     ...    ...       ...      ...       ...      ...
946      Farmer    51.0   36.0      25.0     68.0      36.0     97.0
948         NaN    60.0   23.0      30.0     62.0       NaN      NaN
1004        NaN   237.0   87.0     277.0     64.0     166.0     54.0
1005    Esquire   196.0   63.0      98.0     31.0     223.0     92.0
1006    Esquire   249.0   90.0     124.0     95.0     144.0      0.0

[885 rows x 7 columns]


In [36]:
occs_and_debt = occs_and_debt.dropna(subset=['Occupation'])
print(occs_and_debt)

     Occupation  Dollar  Cents  Dollar.1  Cents.1  Dollar.2  Cents.2
0      Merchant  1064.0   75.0     532.0     37.0     508.0     51.0
1        Farmer   449.0   96.0     224.0     97.0     232.0     10.0
3      Merchant   196.0   75.0      98.0     37.0     172.0     24.0
4      Merchant    53.0   58.0      26.0     79.0      67.0      6.0
6        Farmer   121.0   22.0      60.0     61.0      32.0     72.0
...         ...     ...    ...       ...      ...       ...      ...
939      Trader   387.0   23.0     196.0     85.0       NaN      NaN
944      Farmer   258.0   25.0     129.0     13.0      69.0     72.0
946      Farmer    51.0   36.0      25.0     68.0      36.0     97.0
1005    Esquire   196.0   63.0      98.0     31.0     223.0     92.0
1006    Esquire   249.0   90.0     124.0     95.0     144.0      0.0

[464 rows x 7 columns]


In [37]:
occurences_dict = occs_and_debt['Occupation'].value_counts().to_dict()
print(occurences_dict)

{'Farmer': 164, 'Merchant': 58, 'Trader': 41, 'Gentleman': 32, 'Widow': 25, 'Attorney at Law': 17, 'Mariner': 13, 'Esquire': 11, 'Spinster': 6, 'Shoemaker': 6, 'School Committee': 4, 'Treasurer': 4, 'Attorney': 3, 'Physician': 3, 'Post Rider': 3, 'Traders': 3, 'Inn Keeper': 3, 'Broker': 3, 'Sherriffs Deputy': 3, 'Blacksmith': 3, 'School Master': 2, 'Treasurer to the Society of Cincinnati in  Connecticut': 2, 'Society Treasurer': 2, 'Administrator to J McCurdy': 2, 'Cashier of Hartford Bank': 2, 'Printer': 2, 'Executor to the Estate of Tim Mather Deceased': 1, 'Society Committee': 1, 'Adminastrator of the Asa Waterman': 1, 'Tanner': 1, 'Committee of the 1st Ecclestical Society': 1, 'Molster': 1, 'Administrator to the estate of Daniel Unkus': 1, 'Soldier': 1, 'Executor to Ezra Selden of Lyme Deceased': 1, 'Executor to Stephen Johnson Deceased': 1, 'Hatter': 1, 'Executors to the Estate of Jabez Hamlin Deceased   ': 1, 'Adminastrator to the Estate of Samuel Tilley': 1, 'Treasurer in 1st So

In [38]:
totals = occs_and_debt.groupby('Occupation')[['Dollar', 'Dollar.1', 'Dollar.2', 'Cents', 'Cents.1', 'Cents.2']].apply(sum)
totals = totals.reset_index()
print(totals.to_string())

                                                           Occupation   Dollar  Dollar.1  Dollar.2   Cents  Cents.1  Cents.2
0                                   Adminastrator of the Asa Waterman    218.0     109.0     191.0    47.0     24.0      0.0
1                                      Adminastrator to David Fellows    560.0     280.0       0.0     0.0      0.0      0.0
2   Adminastrator to the Estate of Epaphras Bull of Hartford Deceased     61.0      30.0      16.0    30.0     66.0      5.0
3                        Adminastrator to the Estate of Samuel Tilley     94.0      47.0     118.0    67.0     33.0     99.0
4            Adminastrator to the estate of Uriah Loomis Jun Deceased     73.0      36.0      52.0    50.0     74.0     91.0
5                                          Administrator to J McCurdy   3413.0    1706.0    3785.0    72.0     85.0      0.0
6                         Administrator to the estate of Daniel Unkus     54.0      27.0      51.0     5.0      2.0     86.0


In [39]:
#occupations = occs_and_debt.drop_duplicates(subset=['Occupation']) #remove any duplicate occupation
#occupations = occupations.reset_index(drop=True) #reindex
#occs_debt_wo_dups = occupations.copy() #copy current iteration of occupations. will need later
#occs_debt_wo_dups['Occurences'] = 0 #add occurences column for each occupation. set default value to 0
occs_debt_wo_dups = totals.copy()
#use fuzzy string matching to remove any similar names like merchant and merchents. does not take into account longer titles.
occ_list = occs_debt_wo_dups['Occupation'].tolist()

for occ in occ_list:
    for occ_2 in occ_list:
        if occ != occ_2:
            if rapidfuzz.fuzz.ratio(occ, occ_2) >= 80 and occ != occ_2:
                #transfer amount in row that will be deleted to other row
                row_w_occ_2 = occs_debt_wo_dups.loc[occs_debt_wo_dups['Occupation'] == occ_2] #locate row that contains occupation soon to be deleted
                dollars_in_occ_2 = row_w_occ_2[['Dollar', 'Dollar.1', 'Dollar.2']].sum(axis=1).values #get total dollars
                cents_in_occ_2 = row_w_occ_2[['Cents', 'Cents.1', 'Cents.2']].sum(axis=1).values #get total cents
                dollars_in_occ_2 = dollars_in_occ_2[0] #stored in array. need to retrieve value
                cents_in_occ_2 = cents_in_occ_2[0] #stored in array. need to retrieve value  
                occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Dollar'] += dollars_in_occ_2 #add dollars to occupation 1
                occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Cents'] += cents_in_occ_2 #add cents to occupation 1
                 
                #remove value from occurences dictionary and add one to other occupation
                occurences_dict[occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Occupation'].values[0]] += occurences_dict[occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ_2), 'Occupation'].values[0]] 
                occurences_dict.pop(occ_2)
                
                #remove row
                occs_debt_wo_dups.drop(occs_debt_wo_dups[occs_debt_wo_dups['Occupation'] == occ_2].index, inplace=True) 
                occ_list.remove(occ_2)

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-65 

#take the first two words of every job if applicable and remove any duplicate titles. handles longer titles 
two_word_occs = {}
#retrieve all two worded occupations
for occ in occ_list:
    if len(occ.split()) >= 2: #only account for words that are actually more than two words long
        occ_two_word = occ.split()[0] + " " + occ.split()[1] #get first two words
        two_word_occs[occ_list.index(occ)] = occ_two_word #append both words to dict

#retrieve any duplicates now 
flipped = {} 
for key, value in two_word_occs.items():
    flipped.setdefault(value, list()).append(key) #flip keys with values

keys_w_dups = [key for key, values in flipped.items() if len(values) > 1] #retrieve keys that have multiple values. store them in a list

print(occurences_dict)

{'Farmer': 164, 'Merchant': 59, 'Trader': 44, 'Gentleman': 32, 'Widow': 25, 'Attorney at Law': 17, 'Mariner': 13, 'Esquire': 11, 'Spinster': 6, 'Shoemaker': 6, 'School Committee': 4, 'Treasurer': 4, 'Attorney': 3, 'Physician': 3, 'Post Rider': 3, 'Inn Keeper': 3, 'Broker': 3, 'Sherriffs Deputy': 3, 'Blacksmith': 3, 'School Master': 2, 'Treasurer to the Society of Cincinnati in  Connecticut': 2, 'Society Treasurer': 2, 'Administrator to J McCurdy': 2, 'Cashier of Hartford Bank': 2, 'Printer': 2, 'Executor to the Estate of Tim Mather Deceased': 2, 'Society Committee': 1, 'Adminastrator of the Asa Waterman': 1, 'Tanner': 1, 'Committee of the 1st Ecclestical Society': 1, 'Molster': 1, 'Administrator to the estate of Daniel Unkus': 1, 'Soldier': 1, 'Executor to Ezra Selden of Lyme Deceased': 1, 'Executor to Stephen Johnson Deceased': 2, 'Hatter': 1, 'Executors to the Estate of Jabez Hamlin Deceased   ': 1, 'Adminastrator to the Estate of Samuel Tilley': 1, 'Treasurer in 1st Society': 2, 'Go

In [40]:
print(occs_debt_wo_dups.to_string())

                                                           Occupation   Dollar  Dollar.1  Dollar.2   Cents  Cents.1  Cents.2
0                                   Adminastrator of the Asa Waterman    218.0     109.0     191.0    47.0     24.0      0.0
1                                      Adminastrator to David Fellows    560.0     280.0       0.0     0.0      0.0      0.0
2   Adminastrator to the Estate of Epaphras Bull of Hartford Deceased     61.0      30.0      16.0    30.0     66.0      5.0
3                        Adminastrator to the Estate of Samuel Tilley     94.0      47.0     118.0    67.0     33.0     99.0
4            Adminastrator to the estate of Uriah Loomis Jun Deceased     73.0      36.0      52.0    50.0     74.0     91.0
5                                          Administrator to J McCurdy   3413.0    1706.0    3785.0    72.0     85.0      0.0
6                         Administrator to the estate of Daniel Unkus     54.0      27.0      51.0     5.0      2.0     86.0


In [41]:
print(flipped)
print("\n\n")
print(keys_w_dups)

{'Adminastrator of': [0], 'Adminastrator to': [1, 2, 3, 4], 'Administrator to': [5, 6], 'Adminstrator to': [7], 'Attorney at': [10], 'Cashier of': [14], 'Committee of': [16, 17], 'Executor to': [20, 21, 22, 23], 'Executors to': [24], 'Guardian to': [28, 29, 30], 'Inn Keeper': [32], 'Leather Dresser': [34], 'Post Rider': [40], 'School Committee': [44, 45, 46], 'School Master': [47], 'Sherriffs Deputy': [48], 'Society Committee': [51], 'Society Treasurer': [52], 'Treasurer 1st': [59], 'Treasurer Stepney': [60], 'Treasurer in': [61], 'Treasurer of': [62], 'Treasurer to': [63], 'Wife of': [65]}



['Adminastrator to', 'Administrator to', 'Committee of', 'Executor to', 'Guardian to', 'School Committee']


In [42]:
print(occurences_dict)

for occ in keys_w_dups:
    i = 0
    while i < len(flipped[occ]):
        index = flipped[occ][i]
        if len(flipped[occ]) > 1: #make sure to keep one index in the list. don't want to delete ALL occurences
            duplicate_row = occs_debt_wo_dups.loc[[index]] #return row that has index
            dollars_in_dup_row = duplicate_row[['Dollar', 'Dollar.1', 'Dollar.2']].sum(axis=1).values #sum all dollars in row                
            cents_in_dup_row = duplicate_row[['Cents', 'Cents.1', 'Cents.2']].sum(axis=1).values #return total cents in row
            dollars_dup_row = dollars_in_dup_row[0] #stored in array. need to retrieve value
            cents_dup_row = cents_in_dup_row[0] #stored in array. need to retrieve value

            last_index = flipped[occ][len(flipped[occ]) - 1] #get last value in list. we will add dollars and cents here
                        
            occs_debt_wo_dups.loc[last_index, 'Dollar'] += dollars_dup_row #add dollars from one occupation to the other
            occs_debt_wo_dups.loc[last_index, 'Cents'] += cents_dup_row #add cents from one occupation to the other
            
            #remove value from occurences dictionary and add one to other occupation
            
            #print("Removed: " + str(occs_debt_wo_dups.loc[index, 'Occupation']))
            #print('Kept: ' + str(occs_debt_wo_dups.loc[last_index, 'Occupation']))
            
            print(occs_debt_wo_dups.loc[last_index, 'Occupation'])
            occurences_dict[occs_debt_wo_dups.loc[last_index, 'Occupation']] += occurences_dict[occs_debt_wo_dups.loc[index, 'Occupation']] 
            occurences_dict.pop(occs_debt_wo_dups.loc[index, 'Occupation'])
            
            print("Length: " + str(len(flipped[occ])))
            print("Last Index: " + str(last_index) + " Index: " + str(index))
            print(flipped[occ])
            print("\n\n")
                        
            occs_debt_wo_dups.drop(index, inplace=True) #remove duplicate occupation now
            flipped[occ].remove(index) #remove from list
            
            #print(occurences_dict)
            #print("\n\n")
        else:
            i += 1

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-...

{'Farmer': 164, 'Merchant': 59, 'Trader': 44, 'Gentleman': 32, 'Widow': 25, 'Attorney at Law': 17, 'Mariner': 13, 'Esquire': 11, 'Spinster': 6, 'Shoemaker': 6, 'School Committee': 4, 'Treasurer': 4, 'Attorney': 3, 'Physician': 3, 'Post Rider': 3, 'Inn Keeper': 3, 'Broker': 3, 'Sherriffs Deputy': 3, 'Blacksmith': 3, 'School Master': 2, 'Treasurer to the Society of Cincinnati in  Connecticut': 2, 'Society Treasurer': 2, 'Administrator to J McCurdy': 2, 'Cashier of Hartford Bank': 2, 'Printer': 2, 'Executor to the Estate of Tim Mather Deceased': 2, 'Society Committee': 1, 'Adminastrator of the Asa Waterman': 1, 'Tanner': 1, 'Committee of the 1st Ecclestical Society': 1, 'Molster': 1, 'Administrator to the estate of Daniel Unkus': 1, 'Soldier': 1, 'Executor to Ezra Selden of Lyme Deceased': 1, 'Executor to Stephen Johnson Deceased': 2, 'Hatter': 1, 'Executors to the Estate of Jabez Hamlin Deceased   ': 1, 'Adminastrator to the Estate of Samuel Tilley': 1, 'Treasurer in 1st Society': 2, 'Go

In [43]:
print(occs_debt_wo_dups.to_string())

                                                  Occupation   Dollar  Dollar.1  Dollar.2   Cents  Cents.1  Cents.2
0                          Adminastrator of the Asa Waterman    218.0     109.0     191.0    47.0     24.0      0.0
1   Adminastrator to the estate of Uriah Loomis Jun Deceased   1279.0      36.0      52.0   350.0     74.0     91.0
2                Administrator to the estate of Daniel Unkus   8958.0      27.0      51.0   162.0      2.0     86.0
3       Adminstrator to the Estate of Ezekiel Carey Deceased    323.0      38.0     106.0   294.0     81.0     17.0
4                                                 Apothecary      7.0       3.0       0.0    72.0     86.0      0.0
5                                                   Attorney   4862.0    2430.0    3309.0   167.0    184.0     45.0
6                                            Attorney at Law   6443.0    2804.0    3312.0   793.0    568.0    486.0
7                                                 Blacksmith    723.0   

In [44]:
print(flipped)

{'Adminastrator of': [0], 'Adminastrator to': [4], 'Administrator to': [6], 'Adminstrator to': [7], 'Attorney at': [10], 'Cashier of': [14], 'Committee of': [17], 'Executor to': [23], 'Executors to': [24], 'Guardian to': [30], 'Inn Keeper': [32], 'Leather Dresser': [34], 'Post Rider': [40], 'School Committee': [46], 'School Master': [47], 'Sherriffs Deputy': [48], 'Society Committee': [51], 'Society Treasurer': [52], 'Treasurer 1st': [59], 'Treasurer Stepney': [60], 'Treasurer in': [61], 'Treasurer of': [62], 'Treasurer to': [63], 'Wife of': [65]}


In [45]:
print(occurences_dict)

{'Farmer': 164, 'Merchant': 59, 'Trader': 44, 'Gentleman': 32, 'Widow': 25, 'Attorney at Law': 17, 'Mariner': 13, 'Esquire': 11, 'Spinster': 6, 'Shoemaker': 6, 'Treasurer': 4, 'Attorney': 3, 'Physician': 3, 'Post Rider': 3, 'Inn Keeper': 3, 'Broker': 3, 'Sherriffs Deputy': 3, 'Blacksmith': 3, 'School Master': 2, 'Treasurer to the Society of Cincinnati in  Connecticut': 2, 'Society Treasurer': 2, 'Cashier of Hartford Bank': 2, 'Printer': 2, 'Executor to the Estate of Tim Mather Deceased': 6, 'Society Committee': 1, 'Adminastrator of the Asa Waterman': 1, 'Tanner': 1, 'Molster': 1, 'Administrator to the estate of Daniel Unkus': 3, 'Soldier': 1, 'Hatter': 1, 'Executors to the Estate of Jabez Hamlin Deceased   ': 1, 'Treasurer in 1st Society': 2, 'Goldsmith': 1, 'Leather Dresser': 1, 'Wife of Eliphalet Williams': 1, 'Miller': 1, 'Apothecary': 1, 'Clothier': 1, 'Treasurer 1st Society in Cornwall': 1, 'Joiner': 1, 'Treasurer of Yale College': 1, 'Proprietors': 1, 'Treasurer Stepney Society':

In [46]:
exceptions = ['Society Committee', 'School Committee North Parish']

In [47]:
#get the first word from every occupation and store any duplicates in a dictionary
occ_list = occs_debt_wo_dups['Occupation'].to_list()
occupations = []
one_word_occs = {}
flipped_one_w = {}

for occ in occ_list:
    if occ not in exceptions:
        first_word = occ.split()[0]
        one_word_occs[occ_list.index(occ)] = first_word 
        
#flip dictionary
for key, value in one_word_occs.items():
    flipped_one_w.setdefault(value, list()).append(key) #flip keys with values

In [48]:
#use fuzzy string matching 
flipped_list = list(flipped_one_w) #convert to list. needed in order to iterate through
for occ in flipped_list:
    for occ_2 in flipped_list:
        if occ_2 != occ and rapidfuzz.fuzz.ratio(occ, occ_2) >= 80:
            flipped_one_w[occ].extend(flipped_one_w[occ_2]) #combine lists
            flipped_one_w.pop(occ_2) #remove second occurence of the same occupation 
            flipped_list.remove(occ_2) #remove from list 
            
keys_w_dups = [key for key, values in flipped_one_w.items() if len(values) > 1] #retrieve keys that have multiple values. store them in a list
         
print(flipped_one_w)
print("\n\n")
print(keys_w_dups)

{'Adminstrator': [3, 0, 1, 2], 'Apothecary': [4], 'Attorney': [5, 6], 'Blacksmith': [7], 'Broker': [8], 'Carpenter': [9], 'Cashier': [10], 'Clothier': [11], 'Committee': [12], 'Cooper': [13], 'Esquire': [14], 'Executor': [15, 16], 'Farmer': [17], 'Gentleman': [18], 'Goldsmith': [19], 'Guardian': [20], 'Hatter': [21], 'Inn': [22], 'Joiner': [23], 'Leather': [24], 'Mariner': [25], 'Merchant': [26], 'Miller': [27], 'Molster': [28], 'Physician': [29], 'Post': [30], 'Printer': [31], 'Proprietors': [32], 'Sadler': [33], 'School': [35], 'Sherriffs': [36], 'Shipwright': [37], 'Shoemaker': [38], 'Society': [40], 'Soldier': [41], 'Spinster': [42], 'Tanner': [43], 'Taylor': [44], 'Trader': [45], 'Treasurer': [46, 47, 48, 49, 50, 51], 'Widow': [52], 'Wife': [53]}



['Adminstrator', 'Attorney', 'Executor', 'Treasurer']


In [49]:
#remove duplicates
for occ in keys_w_dups:
    i = 0
    while i < len(flipped_one_w[occ]):
        if len(flipped_one_w[occ]) > 1:
            
            index = flipped_one_w[occ][i]
            last_index = flipped_one_w[occ][len(flipped_one_w[occ]) - 1] #get last value in list. we will add dollars and cents here
            duplicate_row = occs_debt_wo_dups.loc[[index]] #returns row that has index
            dollars_in_dup_row = duplicate_row[['Dollar', 'Dollar.1', 'Dollar.2']].sum(axis=1).values[0]
            cents_in_dup_row = duplicate_row[['Cents', 'Cents.1', 'Cents.2']].sum(axis=1).values[0]
            
            occs_debt_wo_dups.loc[last_index, 'Dollar'] += dollars_in_dup_row
            occs_debt_wo_dups.loc[last_index, 'Cents'] += cents_dup_row
            
            occurences_dict[occs_debt_wo_dups.loc[last_index, 'Occupation']] += occurences_dict[occs_debt_wo_dups.loc[index, 'Occupation']] 
            occurences_dict.pop(occs_debt_wo_dups.loc[index, 'Occupation'])

            occs_debt_wo_dups.drop(index, inplace=True)
            flipped_one_w[occ].remove(index)
            
        else:
            i += 1

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-...

print(occs_debt_wo_dups.to_string())

                                                Occupation   Dollar  Dollar.1  Dollar.2   Cents  Cents.1  Cents.2
0              Administrator to the estate of Daniel Unkus  11310.0      27.0      51.0   960.0      2.0     86.0
1                                               Apothecary      7.0       3.0       0.0    72.0     86.0      0.0
2                                          Attorney at Law  17044.0    2804.0    3312.0  1059.0    568.0    486.0
3                                               Blacksmith    723.0     360.0     463.0   171.0    235.0    179.0
4                                                   Broker    228.0     114.0     441.0    55.0     28.0    178.0
5                                                Carpenter    175.0      87.0      63.0    34.0     67.0     12.0
6                                 Cashier of Hartford Bank     85.0      42.0      67.0    30.0     66.0     48.0
7                                                 Clothier     53.0      26.0      38.0 

In [50]:
occs_debt_wo_dups[['Cents', 'Cents.1', 'Cents.2']] = occs_debt_wo_dups[['Cents', 'Cents.1', 'Cents.2']].div(100)
print(occs_debt_wo_dups.to_string())

                                                Occupation   Dollar  Dollar.1  Dollar.2  Cents  Cents.1  Cents.2
0              Administrator to the estate of Daniel Unkus  11310.0      27.0      51.0   9.60     0.02     0.86
1                                               Apothecary      7.0       3.0       0.0   0.72     0.86     0.00
2                                          Attorney at Law  17044.0    2804.0    3312.0  10.59     5.68     4.86
3                                               Blacksmith    723.0     360.0     463.0   1.71     2.35     1.79
4                                                   Broker    228.0     114.0     441.0   0.55     0.28     1.78
5                                                Carpenter    175.0      87.0      63.0   0.34     0.67     0.12
6                                 Cashier of Hartford Bank     85.0      42.0      67.0   0.30     0.66     0.48
7                                                 Clothier     53.0      26.0      38.0   0.33  

In [51]:
occs_debt_wo_dups['Total'] = occs_debt_wo_dups[['Dollar', 'Dollar.1', 'Dollar.2', 'Cents', 'Cents.1', 'Cents.2']].sum(axis=1) #add totals to new column
print(occs_debt_wo_dups.to_string())

                                                Occupation   Dollar  Dollar.1  Dollar.2  Cents  Cents.1  Cents.2      Total
0              Administrator to the estate of Daniel Unkus  11310.0      27.0      51.0   9.60     0.02     0.86   11398.48
1                                               Apothecary      7.0       3.0       0.0   0.72     0.86     0.00      11.58
2                                          Attorney at Law  17044.0    2804.0    3312.0  10.59     5.68     4.86   23181.13
3                                               Blacksmith    723.0     360.0     463.0   1.71     2.35     1.79    1551.85
4                                                   Broker    228.0     114.0     441.0   0.55     0.28     1.78     785.61
5                                                Carpenter    175.0      87.0      63.0   0.34     0.67     0.12     326.13
6                                 Cashier of Hartford Bank     85.0      42.0      67.0   0.30     0.66     0.48     195.44
7       

In [52]:
occs_debt_wo_dups['# of People'] = occs_debt_wo_dups['Occupation'].map(occurences_dict) #add occurences to table
print(occs_debt_wo_dups[['Occupation', '# of People']])

                                           Occupation  # of People
0         Administrator to the estate of Daniel Unkus           10
1                                          Apothecary            1
2                                     Attorney at Law           20
3                                          Blacksmith            3
4                                              Broker            3
5                                           Carpenter            1
6                            Cashier of Hartford Bank            2
7                                            Clothier            1
8          Committee of the 1st Society of Plainville            2
9                                              Cooper            1
10                                            Esquire           11
11  Executors to the Estate of Jabez Hamlin Deceas...            7
12                                             Farmer          164
13                                          Gentleman         

In [53]:
occs_debt_wo_dups['Average'] = occs_debt_wo_dups['Total'] / occs_debt_wo_dups['# of People'] #calculate average and add to new column
print(occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average']].to_string())

                                                Occupation      Total  # of People      Average
0              Administrator to the estate of Daniel Unkus   11398.48           10  1139.848000
1                                               Apothecary      11.58            1    11.580000
2                                          Attorney at Law   23181.13           20  1159.056500
3                                               Blacksmith    1551.85            3   517.283333
4                                                   Broker     785.61            3   261.870000
5                                                Carpenter     326.13            1   326.130000
6                                 Cashier of Hartford Bank     195.44            2    97.720000
7                                                 Clothier     118.40            1   118.400000
8               Committee of the 1st Society of Plainville     932.38            2   466.190000
9                                       

In [54]:
#convert to one word
exceptions = ['Society Committee', 'Society Treasurer', 'School Master', 'Inn Keeper', 'Leather Dresser', 'Post Rider', 'School Committee North Parish']
def first_word_only(occ):
    if 'School Committee' in occ:
        return 'School Committee'
    if occ not in exceptions:
        return occ.split()[0]
    else:
        return occ
    
occs_debt_wo_dups['Occupation'] = occs_debt_wo_dups['Occupation'].apply(first_word_only)
print(occs_debt_wo_dups.to_string())

           Occupation   Dollar  Dollar.1  Dollar.2  Cents  Cents.1  Cents.2      Total  # of People      Average
0       Administrator  11310.0      27.0      51.0   9.60     0.02     0.86   11398.48           10  1139.848000
1          Apothecary      7.0       3.0       0.0   0.72     0.86     0.00      11.58            1    11.580000
2            Attorney  17044.0    2804.0    3312.0  10.59     5.68     4.86   23181.13           20  1159.056500
3          Blacksmith    723.0     360.0     463.0   1.71     2.35     1.79    1551.85            3   517.283333
4              Broker    228.0     114.0     441.0   0.55     0.28     1.78     785.61            3   261.870000
5           Carpenter    175.0      87.0      63.0   0.34     0.67     0.12     326.13            1   326.130000
6             Cashier     85.0      42.0      67.0   0.30     0.66     0.48     195.44            2    97.720000
7            Clothier     53.0      26.0      38.0   0.33     0.67     0.40     118.40          

In [55]:
total = occs_debt_wo_dups['Total'].sum()
occs_debt_wo_dups['% of Total Debt'] = round((occs_debt_wo_dups['Total'] / total) * 100, 3)
print(occs_debt_wo_dups.to_string())

           Occupation   Dollar  Dollar.1  Dollar.2  Cents  Cents.1  Cents.2      Total  # of People      Average  % of Total Debt
0       Administrator  11310.0      27.0      51.0   9.60     0.02     0.86   11398.48           10  1139.848000            2.454
1          Apothecary      7.0       3.0       0.0   0.72     0.86     0.00      11.58            1    11.580000            0.002
2            Attorney  17044.0    2804.0    3312.0  10.59     5.68     4.86   23181.13           20  1159.056500            4.991
3          Blacksmith    723.0     360.0     463.0   1.71     2.35     1.79    1551.85            3   517.283333            0.334
4              Broker    228.0     114.0     441.0   0.55     0.28     1.78     785.61            3   261.870000            0.169
5           Carpenter    175.0      87.0      63.0   0.34     0.67     0.12     326.13            1   326.130000            0.070
6             Cashier     85.0      42.0      67.0   0.30     0.66     0.48     195.44    

In [56]:
#final total debt held by people w/ no job listed
occs_and_debt = pd.read_excel(r"C:\Users\david\OneDrive\Documents\GitHub\SPEOC-pt-1\Data\Post1790\CT\CT_post1790_CD_ledger.xlsx",
                             header=13, usecols='L, N, O, AD, AE, AT, AU') #occupations plus continental debt for conneticut
no_occ = occs_and_debt[occs_and_debt['Occupation'].isna()]
num_of_people = len(no_occ)
no_occ = no_occ.reset_index(drop=True)
no_occ[['Cents', 'Cents.1', 'Cents.2']] = no_occ[['Cents', 'Cents.1', 'Cents.2']].div(100)
total_series = no_occ[['Dollar', 'Dollar.1', 'Dollar.2', 'Cents', 'Cents.1', 'Cents.2']].sum(axis=1)
total = total_series.sum()

occs_debt_wo_dups = occs_debt_wo_dups.append({'Occupation': 'None Listed', 'Total': total, '# of People': num_of_people, 'Average': total / num_of_people}, ignore_index=True)
print(occs_debt_wo_dups.to_string())

           Occupation   Dollar  Dollar.1  Dollar.2  Cents  Cents.1  Cents.2       Total  # of People      Average  % of Total Debt
0       Administrator  11310.0      27.0      51.0   9.60     0.02     0.86    11398.48           10  1139.848000            2.454
1          Apothecary      7.0       3.0       0.0   0.72     0.86     0.00       11.58            1    11.580000            0.002
2            Attorney  17044.0    2804.0    3312.0  10.59     5.68     4.86    23181.13           20  1159.056500            4.991
3          Blacksmith    723.0     360.0     463.0   1.71     2.35     1.79     1551.85            3   517.283333            0.334
4              Broker    228.0     114.0     441.0   0.55     0.28     1.78      785.61            3   261.870000            0.169
5           Carpenter    175.0      87.0      63.0   0.34     0.67     0.12      326.13            1   326.130000            0.070
6             Cashier     85.0      42.0      67.0   0.30     0.66     0.48      19

In [57]:
occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average', '% of Total Debt']].to_excel("./FinishedSpreadsheets/occupation_debt_" + state + ".xlsx")